In [1]:
import numpy as np
import pandas as pd

In [2]:
cases = pd.read_csv("C:/Users/shrof/Downloads/data100k.csv")

In [3]:
cases.head(3).T

,0,1,2
person_id,102090000000110,343221000000125,343221000000125
HearingDate,2019-02-28,2009-12-07,2011-01-20
CodeSection,A.46.2-862,B.46.2-301,A.46.2-707
codesection,covered elsewhere,covered elsewhere,covered elsewhere
ChargeType,Misdemeanor,Misdemeanor,Misdemeanor
chargetype,Misdemeanor,Misdemeanor,Misdemeanor
Class,1,1,3
DispositionCode,Guilty,Guilty,Guilty
disposition,Conviction,Conviction,Conviction
Plea,NaN,NaN,NaN


In [ ]:
## What code sections are most frequent? 

In [8]:
codesections = cases['CodeSection'].value_counts().reset_index()
codesections.head(15)

,CodeSection,count
0,A.46.2-862,26379
1,B.46.2-301,25967
2,46.2-300,17934
3,C.46.2-862,11728
4,18.2-250.1,10573
5,A.18.2-266,8568
6,18.2-95,7561
7,18.2-250,6949
8,18.2-57,6699
9,A.46.2-852,6667


### Which codes lead to most convictions? 

In [12]:
cases['DispositionCode'].value_counts()
cases['conviction'] = [x in ['Guilty', 'Guilty In Abstentia'] for x in cases['DispositionCode']]

In [19]:
'''
SELECT CodeSection, AVG(conviction) AS convict_rate
FROM CASES
GROUP BY CodeSection
'''

convict_rate = cases.groupby('CodeSection').agg({'conviction':['mean','count']}).reset_index()

In [25]:
convict_rate.columns = ['CodeSection', 'conviction_rate', 'count']
convict_rate = convict_rate.query("count >= 30")
convict_rate.sort_values('conviction_rate', ascending=False)

,CodeSection,conviction_rate,count
1633,21-336,0.960000,50
737,18.2-195(1)(A),0.926829,41
1538,21-1/46.2-301,0.913043,46
4111,G.18.2-266,0.906977,43
4012,B.46.2-357,0.904891,736
...,...,...,...
2690,41.1-2-2,0.015504,129
253,13-60,0.014286,70
1433,19.2-100,0.000000,238
140,11.1-2,0.000000,38


In [27]:
cases.query("CodeSection == '23-55'")('fips')

TypeError: 'DataFrame' object is not callable

### Most racial disparities

In [29]:
# filter out ones with small counts
cases['Race'].unique()

array(['Black(Non-Hispanic)', 'Hispanic', 'White Caucasian(Non-Hispanic)',
       'MISSING', 'Asian Or Pacific Islander', 'Black (Non-Hispanic)',
       'White Caucasian (Non-Hispanic)',
       'Other(Includes Not Applicable.. Unknown)',
       'Other (Includes Not Applicable.. Unknown)', 'Black', 'White',
       'Unknown (Includes Not Applicable.. Unknown)', 'American Indian',
       'Unknown', 'Asian or Pacific Islander',
       'American Indian Or Alaskan Native'], dtype=object)

In [30]:
replace_map = {'Black(Non-Hispanic)':'Black',
'Hispanic':'Hispanic', 
'White Caucasian(Non-Hispanic)':'White',
'MISSING':'Missing/Other', 
'Asian Or Pacific Islander':'Asian or Pacific Islander',
'Black (Non-Hispanic)':'Black',
'White Caucasian (Non-Hispanic)':'White',
'Other(Includes Not Applicable.. Unknown)':'Missing/Other',
'Other (Includes Not Applicable.. Unknown)':'Missing/Other',
'Black':'Black', 
'White':'White',
'Unknown (Includes Not Applicable.. Unknown)':'Missing/Other',
'American Indian':'American Indian or Alaskan Native',
'Unknown':'', 
'Asian or Pacific Islander':'Asian or Pacific Islander',
'American Indian Or Alaskan Native':'American Indian or Alaskan Native'}

cases['Race'] = cases['Race'].replace(replace_map)
cases['Race'].value_counts()

Race
White                                159627
Black                                115627
Hispanic                               9319
Missing/Other                          5874
Asian or Pacific Islander              2794
American Indian or Alaskan Native       303
                                         54
Name: count, dtype: int64

In [ ]:
# I choose to analyze only the convictions

In [35]:
cases_convict = cases.query("conviction == True")
cases_convict_race = cases_convict.groupby(['CodeSection', 'Race']).size().reset_index()
cases_convict_race = cases_convict_race.rename({0:'count'}, axis=1)
cases_convict_race

,CodeSection,Race,count
0,01-2007,White,1
1,1,Black,1
2,1,White,1
3,1-12,Black,24
4,1-12,White,5
...,...,...,...
4261,Z.18.2-91,Hispanic,2
4262,Z.18.2-91,White,123
4263,Z.18.2-91; 26,Black,1
4264,Z.18.2-95,Black,2


In [42]:
cases_reshape = cases_convict_race.pivot_table(index = 'CodeSection', columns = 'Race', 
                                               values = 'count', fill_value=0).reset_index()
cases_reshape

Race,CodeSection,,American Indian or Alaskan Native,Asian or Pacific Islander,Black,Hispanic,Missing/Other,White
0,01-2007,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,1-12,0.0,0.0,0.0,24.0,0.0,0.0,5.0
3,1-200,0.0,0.0,0.0,7.0,0.0,0.0,3.0
4,1.21,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...
2672,Z.18.2-90,0.0,0.0,0.0,4.0,0.0,1.0,3.0
2673,Z.18.2-91,0.0,0.0,2.0,95.0,2.0,0.0,123.0
2674,Z.18.2-91; 26,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2675,Z.18.2-95,0.0,0.0,0.0,2.0,0.0,0.0,0.0


In [46]:
cases_reshape = cases_reshape.assign(total=cases_reshape['American Indian or Alaskan Native'] + 
                                              cases_reshape['Asian or Pacific Islander'] +
                                              cases_reshape['Black'] + 
                                              cases_reshape['Hispanic'] + 
                                              cases_reshape['Missing/Other'] + 
                                              cases_reshape['White'])

In [ ]:
cases_reshape = cases_reshape.query("total > 50")

In [49]:
cases_reshape = cases_reshape.assign(black_percent = cases_reshape['Black']/cases_reshape['total'],
                                     white_percent = cases_reshape['White']/cases_reshape['total'])


In [50]:
cases_reshape.sort_values('black_percent', ascending=False)

Race,CodeSection,,American Indian or Alaskan Native,Asian or Pacific Islander,Black,Hispanic,Missing/Other,White,total,black_percent,white_percent
2676,Z18.2-47,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
506,18.2-248A C,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
1327,28-72,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
1326,28-5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
1325,28-43,0.0,0.0,0.0,4.0,0.0,0.0,0.0,4.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
419,18.2-192(B),0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1573,33.1-396,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1572,33.1-369,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1570,33.1-218,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0


### In what localities (fips) are these disparities most severe? 

In [ ]:
cases